In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from environment import *

with open("setting.yaml") as yaml_file:

    SETTING = yaml.load(yaml_file)

PATH = make_path_dict(SETTING)

In [ ]:
if SETTING["signal_type"] == "raw":

    raise

In [ ]:
feature_context = pd.read_table(
    PATH["feature_x_sample.feature_context.tsv"], index_col=0
)

feature_context.columns.name = SETTING["sample_alias"]

In [ ]:
sample_context = pd.read_table(
    PATH["feature_x_sample.sample_context.tsv"], index_col=0
).T

sample_context.index.name = SETTING["feature_alias"]

In [ ]:
feature_context_negative = -feature_context.clip(upper=0)

feature_context_positive = feature_context.clip(lower=0)

In [ ]:
sample_context_negative = -sample_context.clip(upper=0)

sample_context_positive = sample_context.clip(lower=0)

In [ ]:
if "feature" in SETTING["element_types"] and "sample" in SETTING["element_types"]:

    normalize_df_args = (None, "0-1", False)

    context_negative = ccal.normalize_df(
        feature_context_negative, *normalize_df_args
    ) + ccal.normalize_df(sample_context_negative, *normalize_df_args)

    context_positive = ccal.normalize_df(
        feature_context_positive, *normalize_df_args
    ) + ccal.normalize_df(sample_context_positive, *normalize_df_args)

elif "feature" in SETTING["element_types"]:

    context_negative = feature_context_negative

    context_positive = feature_context_positive

elif "sample" in SETTING["element_types"]:

    context_negative = sample_context_negative

    context_positive = sample_context_positive

In [ ]:
title_template = "{} Ranking by Context {} Signal"

value_name_template = "{} Signal Sum"

In [ ]:
if SETTING["select_feature_automatically"]:

    context_negative_features = ccal.select_series_indices(
        context_negative.sum(axis=1),
        ">",
        standard_deviation=SETTING["select_feature_standard_deviation"],
        title=dict(text=title_template.format(SETTING["feature_alias"], "Negative")),
        xaxis=dict(title="Rank"),
        yaxis=dict(title=value_name_template.format("Negative")),
    )

    context_positive_features = ccal.select_series_indices(
        context_positive.sum(axis=1),
        ">",
        standard_deviation=SETTING["select_feature_standard_deviation"],
        title=dict(text=title_template.format(SETTING["feature_alias"], "Positive")),
        xaxis=dict(title="Rank"),
        yaxis=dict(title=value_name_template.format("Positive")),
    )

else:

    context_negative_features = (
        context_negative.replace(0, np.nan).dropna(how="all").index
    )

    context_positive_features = (
        context_positive.replace(0, np.nan).dropna(how="all").index
    )

context_negative_features.size, context_positive_features.size

In [ ]:
if SETTING["select_sample_automatically"]:

    context_negative_samples = ccal.select_series_indices(
        context_negative.sum(axis=1),
        ">",
        standard_deviation=SETTING["select_sample_standard_deviation"],
        title=dict(text=title_template.format(SETTING["sample_alias"], "Negative")),
        xaxis=dict(title="Rank"),
        yaxis=dict(title=value_name_template.format("Negative")),
    )

    context_positive_samples = ccal.select_series_indices(
        context_positive.sum(axis=1),
        ">",
        standard_deviation=SETTING["select_sample_standard_deviation"],
        title=dict(text=title_template.format(SETTING["sample_alias"], "Positive")),
        xaxis=dict(title="Rank"),
        yaxis=dict(title=value_name_template.format("Positive")),
    )

else:

    context_negative_samples = (
        context_negative.replace(0, np.nan).dropna(how="all", axis=1).columns
    )

    context_positive_samples = (
        context_positive.replace(0, np.nan).dropna(how="all", axis=1).columns
    )

context_negative_samples.size, context_positive_samples.size

In [ ]:
context_negative_signal = context_negative.loc[
    context_negative_features, context_negative_samples
]

In [ ]:
context_positive_signal = context_positive.loc[
    context_positive_features, context_positive_samples
]

In [ ]:
if "negative" in SETTING["contexts"] and "positive" in SETTING["contexts"]:

    context_negative_signal.index = pd.Index(
        ("(-) {}".format(feature) for feature in context_negative_signal.index),
        name=context_negative_signal.index.name,
    )

    context_positive_signal.index = pd.Index(
        ("(+) {}".format(feature) for feature in context_positive_signal.index),
        name=context_positive_signal.index.name,
    )

    context_signal = pd.concat((context_negative_signal, context_positive_signal))

elif "negative" in SETTING["contexts"]:

    context_signal = context_negative_signal

elif "positive" in SETTING["contexts"]:

    context_signal = context_positive_signal

In [ ]:
if SETTING["context_signal_normalization_method"] is not None:

    context_signal = ccal.normalize_df(
        context_signal,
        SETTING["context_signal_normalization_axis"],
        SETTING["context_signal_normalization_method"],
        raise_for_bad=False,
    )

In [ ]:
context_signal.to_csv(PATH["feature_x_sample.signal.tsv"], sep="\t")

ccal.summarize_feature_x_sample(
    context_signal,
    feature_x_sample_alias="Context Signal",
    feature_x_sample_value_name="Context Signal",
    plot_heat_map_max_size=SETTING["plot_heat_map_max_size"],
    plot_distributions_max_size=SETTING["plot_distributions_max_size"],
    plot_rug_max_size=SETTING["plot_rug_max_size"],
)